# ***Mini RAG - Glossaire des principaux métiers de la Data***

## *DJEBRIL LAOUEDJ*

## **1. Ingestion**

* *charger (le coprus)*

* *nettoyer (préparation des données)*

* *découper (en chunks)*

* *sauvegarder (Un chunk par ligne)*

## *1.1 - Installation de Langchain*

In [ ]:
# === INSTALLATION AUTOMATIQUE ===
!pip install -q -r requirements.txt

import langchain, langchain_core, langchain_community, langchain_experimental, pydantic
print("Versions installées :")
print("langchain", langchain.__version__)
print("core     ", langchain_core.__version__)
print("community", langchain_community.__version__)
print("pydantic ", pydantic.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.6/443.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.8/449.8 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 20.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph-prebuilt 1.0.5 requires langchain-core>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
Versions installées :
langchain 0.3.27
core      0.3.79
community 0.3.31
pydantic  2.11.3


In [ ]:
try:
    from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
except Exception:
    from langchain.retrievers import ContextualCompressionRetriever  # fallback

CrossEncoderReranker = None
try:
    from langchain_community.document_compressors import CrossEncoderReranker
except Exception:
    try:
        from langchain_experimental.retrievers.document_compressors import CrossEncoderReranker
    except Exception:
        try:
            from langchain.retrievers.document_compressors import CrossEncoderReranker
        except Exception as e:
            raise ImportError(
                "Impossible d'importer CrossEncoderReranker dans cette stack. "
                "Vérifie bien les versions installées ci-dessus."
            ) from e

from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

## *1.2 - Chargement du corpus*

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Syntec-Conseil_Glossaire-des-principaux-métiers-de-la-Data.pdf")
# if you use Google colab
# loader = PyPDFLoader("/content/Syntec-Conseil_Glossaire-des-principaux-métiers-de-la-Data.pdf")
docs = loader.load()

## *1.3 - Découpage du texte en chunks*

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
chunks = splitter.split_documents(docs)

## *1.4 - Affichage d'un aperçu*

In [ ]:
print("Nombre de chunks :", len(chunks))
print("Exemple du quatrième chunk :")
print(chunks[3].page_content[:700])

Nombre de chunks : 22
Exemple du quatrième chunk :
Le Chief Analytics Officer exploite des outils informatiques, 
techniques et utilise des méthodes statistiques (y compris data 
science) pour permettre d’organiser, synthétiser et traduire 
efficacement les données.
Il repère, parmi toutes les informations à disposition de 
l’entreprise, quelles sont les plus importantes / pertinentes à 
extraire pour des prises de décisions optimales, en s’appuyant 
sur une méthodologie objective basée sur les statistiques. Le cas 
échéant, il s’assure que les informations recueillies en interne ou 
en externe sont fiables, cohérentes, et prêtes à être analysées.
Il peut aussi piloter l’industrialisation du procédé pour les 
données les plus intéressantes. 


## **2. Embeddings & Index + Retriever**

## *2.1 - Choix du modèle d'embedding*

*Modèles candidats :*

- *bge-m3*

- *multilingual-e5-small*

- *paraphrase-multilingual-MiniLM-L12-v2*

In [ ]:
modele_embedding = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## *2.2 - Vector store (FAISS)*

*Index qui fait du kNN pour retrouver les chunks proches d’une question.*

In [ ]:
vectorstore = FAISS.from_documents(chunks, modele_embedding)
print("Index FAISS prêt")

Index FAISS prêt


## *2.3 - Test*

*Plus le score est faible → plus le chunk est proche de la requête car sous FAISS, c’est une distance L2, donc plus petit = mieux.*

In [ ]:
q = "Quelles sont les missions d’un Data Engineer ?"
# on récupère les k chunks les plus proches (par similarité cosinus)
# avec leurs scores
docs_with_scores = vectorstore.similarity_search_with_score(q, k=3)

for i, (doc, score) in enumerate(docs_with_scores, 1):
    print(f"\n[{i}]  Score: {round(score, 4)}")
    print(doc.page_content[:400])


[1]  Score: 0.5325000286102295
Référent
Le Data Engineer (Ingénieur Data) développe l’infrastructure 
définie par/avec le Data Architect. Il construit les solutions 
techniques robustes (via des tests de robustesse) et fiables. Il en 
assure la maintenance et les évolutions conformément à l’état 
et des contraintes de sécurité.
Il réalise l’intégration des données de diverses natures qui 
proviennent de ces sources multiples, l

[2]  Score: 0.6524999737739563
technologies 
de pointe
Le Data Scientist traite, analyse et valorise les données d’une
entreprise afin de définir la meilleure stratégie de développement : 
stratégie marketing et commerciale, amélioration des perfor -
mances et de la rentabilité, prospective…
Cumulant la connaissance des outils mathématiques / statistiques  
et informatiques, il est capable de les coder (R, Python), 
de produire

[3]  Score: 0.6944000124931335
Spark SQL) et Hadoop pour la partie Big Data, le Cloud, 
les méthodes DevOps et CRISP.
• Bac+5 obligat

## *2.4 - Petit test MMR (Maximal Marginal Relevance est plus pertinent)*

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 4, "fetch_k": 20, "lambda_mult": 0.5}
)

docs_mmr = retriever.invoke(q)
for i, d in enumerate(docs_mmr, 1):
    print(f"\n[MMR {i}] {d.page_content[:350]}...")


[MMR 1] Référent
Le Data Engineer (Ingénieur Data) développe l’infrastructure 
définie par/avec le Data Architect. Il construit les solutions 
techniques robustes (via des tests de robustesse) et fiables. Il en 
assure la maintenance et les évolutions conformément à l’état 
et des contraintes de sécurité.
Il réalise l’intégration des données de diverses na...

[MMR 2] Glossaire
des principaux 
métiers de 
la data...

[MMR 3] de la conformité et de l’éthique. 
Le DPO est chargé de veiller la conformité au règlement, de 
définir les rôles et responsabilités de chacun, d’établir une 
cartographie des traitements et flux de données, de tenir le 
registre des traitements et de piloter la gestion des incidents 
de sécurité (y compris avec les sous-traitants).
En France, ce r...

[MMR 4] donne 
du sens
Performance
L’artiste 
de la donnée, 
le peintre du tableau 
et qui excelle dans 
la description de 
son oeuvre...
le chef 
de production 
de l’équipe
DataOps Engineer
Le Data Visualisation Co

## **3. Reranker**

*On sélectionne un pool de 20 passages, ensuite avec MMR, on sélectionne 8 passages diversifiés dans ce pool.*

In [ ]:
base_ret = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 8, "fetch_k": 20, "lambda_mult": 0.5})

*On crée un compresseur/reranker basé sur le cross-encoder bge-reranker-v2-m3 de BAAI pour Langchain.*

In [ ]:
ce = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3", model_kwargs={"device": "cuda"})
# top_n = combien de passages garder après rerank (ex: 4)
compressor = CrossEncoderReranker(model=ce, top_n=4)

**Combiner retriever de base et reranker pour avoir ce résultat :**

*Quand on lui demande des docs, on obtient directement les meilleurs après reranking.*

In [ ]:
rerank_ret = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=base_ret)

*Retourne la liste finale de passages les plus pertinents (déjà triés et filtrés) à donner au LLM.*

In [ ]:
q = "Quelles sont les missions d’un Data Engineer ?"
docs_for_llm = rerank_ret.get_relevant_documents(q)

for i, d in enumerate(docs_for_llm, 1):
    print(f"\n[Rerank {i}] source={d.metadata.get('source')} page={d.metadata.get('page')}")
    print(d.page_content[:350])

/tmp/ipython-input-1625506513.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs_for_llm = rerank_ret.get_relevant_documents(q)



[Rerank 1] source=/content/Syntec-Conseil_Glossaire-des-principaux-métiers-de-la-Data.pdf page=4
Référent
Le Data Engineer (Ingénieur Data) développe l’infrastructure 
définie par/avec le Data Architect. Il construit les solutions 
techniques robustes (via des tests de robustesse) et fiables. Il en 
assure la maintenance et les évolutions conformément à l’état 
et des contraintes de sécurité.
Il réalise l’intégration des données de diverses na

[Rerank 2] source=/content/Syntec-Conseil_Glossaire-des-principaux-métiers-de-la-Data.pdf page=5
technologies 
de pointe
Le Data Scientist traite, analyse et valorise les données d’une
entreprise afin de définir la meilleure stratégie de développement : 
stratégie marketing et commerciale, amélioration des perfor -
mances et de la rentabilité, prospective…
Cumulant la connaissance des outils mathématiques / statistiques  
et informatiques, il 

[Rerank 3] source=/content/Syntec-Conseil_Glossaire-des-principaux-métiers-de-la-Data.pdf page=6
donn

## **4. LLM + Prompt**

## *4.1 - Installer et lancer Ollama dans Colab*

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!nohup ollama serve > /dev/null 2>&1 &

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


## *4.2 - Téléchargement du modèle*

In [ ]:
# Choix parmi ces modèles : mistral:instruct / phi3:mini / llama3.2:3b-instruct
MODEL_NAME = "mistral:instruct"
!ollama pull {MODEL_NAME}

Error: ollama server not responding - could not connect to ollama server, run 'ollama serve' to start it


## *4.3 - Test rapide pour vérifier qu’Ollama répond*

(Si cela ne marche pas, il faut relancer la 4.1 et la 4.2)

In [ ]:
import requests, json

def ollama_chat(model, prompt):
    url = "http://127.0.0.1:11434/api/generate"
    data = {"model": model, "prompt": prompt, "stream": False}
    r = requests.post(url, json=data, timeout=120)
    r.raise_for_status()
    return r.json()["response"]

print(ollama_chat(MODEL_NAME, "Dis bonjour en une phrase."))

 Bonjour, c'est moi ! Comment ça va ? (Hello there, it's me! How are you?)


## *4.4 - Intégration LangChain (LLM local Ollama)*

In [ ]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model=MODEL_NAME, temperature=0.2)

## *4.5 - Brancher le LLM Ollama sur le mini RAG existant*

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

prompt = PromptTemplate.from_template(
    "Tu es un assistant des métiers de la Data. "
    "Réponds UNIQUEMENT à partir du CONTEXTE fourni. "
    "Si l'information n'y est pas, dis : 'Je ne sais pas sur la base du document.'\n\n"
    "[CONTEXTE]\n{context}\n\n[QUESTION]\n{question}\n\n"
    "Réponse concise en français :"
)

qa_local = RetrievalQA.from_chain_type(
    llm=llm,                          # Ollama (local)
    retriever=rerank_ret,             # Retriever MMR + BGE-reranker
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

# 5. Test du RAG local

## *5.1 - Test Simple*

In [ ]:
question = "Quelles sont les missions d’un Data Engineer ?"
res = qa_local.invoke({"query": question})

print("Réponse :\n", res["result"])

Réponse :
  Les missions d'un Data Engineer consistent à développer l'infrastructure définie par le Data Architect, construire des solutions techniques robustes et fiables, assurer leur maintenance et évolutions conformément aux contraintes de sécurité. Il réalise l'intégration des données de diverses natures, les supervise et vérifie la qualité des données. En production, il assure le suivi et le monitoring des flux/interfaces de données. Il s'assure aussi que ses travaux sont suffisamment documentés.


## *5.2 - Test Interactif (Style ChatBot)*

In [ ]:
question_utilisateur = input("Entrez une question : ")
res = qa_local.invoke({"query": question_utilisateur})

print("Réponse :\n", res["result"])

Entrez une question : Quel est le rôle d'un Data Analyst ?
Réponse :
  Le rôle d'un Data Analyst consiste à explorer et analyser des données pour identifier des tendances, déterminer les relations entre les données et fournir des informations utiles pour prendre des décisions optimales. Il travaille souvent en lien avec le Data Scientist et le Data Engineer.
